<a href="https://colab.research.google.com/github/ssrbazpur/Data-Science/blob/master/Pysprak%20Random%20Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apache.osuosl.org/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
# 2. Setup Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [0]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/drive/My Drive/Data Science/NYC Taxi-eb10197ea31e.json"
%reload_ext google.cloud.bigquery

In [0]:
import pandas as pd
import numpy as np
import sys
from google.cloud import bigquery
from google.colab import auth
auth.authenticate_user()

In [0]:
project_id = 'nyc-taxi-265120'
from google.cloud import bigquery
client = bigquery.Client(project = project_id)


In [0]:
query = """
with wd as (
    SELECT 
        cast(year as STRING) as year,
        EXTRACT (DAYOFYEAR FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP)) AS daynumber, 
        MIN(EXTRACT (DAYOFWEEK FROM CAST(CONCAT(year,'-',mo,'-',da) AS TIMESTAMP))) dayofweek,
        MIN(min) mintemp, MAX(max) maxtemp, MAX(IF(prcp=99.99,0,prcp)) rain
    FROM `bigquery-public-data.noaa_gsod.gsod2019`
    WHERE stn='725030'   --station id 725030=LaGuardia
    GROUP BY 1,2 
  -- TAXI DATA
  ),
TD AS(
WITH AB AS
(
SELECT  CAST(tpep_pickup_datetime AS DATETIME) AS PICKUP,* FROM `nyc-taxi-265120.NYC.2019_CreatedDataset`
)  
SELECT  PICKUP,CAST(EXTRACT (YEAR from PICKUP) AS STRING) AS year,EXTRACT (DAYOFYEAR from PICKUP) AS daynumber,EXTRACT (HOUR from PICKUP) AS hour, cast(PULocationID as STRING) as LOCATION,COUNT(*) AS numtrips FROM AB group by PICKUP,year,daynumber,hour,LOCATION),points AS
(SELECT *,ST_CENTROID(zone_geom) as p FROM `bigquery-public-data.new_york_taxi_trips.taxi_zone_geom`)
SELECT 
td.daynumber,
td.hour,
cast(wd.dayofweek as STRING) as dayofweek, 
    wd.mintemp, 
    wd.maxtemp,
    wd.rain,  
    SUM(td.numtrips) as label,
    ST_ASTEXT(points.p) AS P, 
    points.zone_id,  
  FROM wd, td INNER JOIN points ON points.zone_id=td.LOCATION
  where wd.year = td.year AND
  wd.daynumber = td.daynumber AND
  eXTRACT (YEAR from PICKUP) =2019
  group by daynumber,hour,zone_id,P,dayofweek, mintemp, maxtemp, rain
"""
df = client.query(query).to_dataframe()


In [11]:
df.head()

,daynumber,hour,dayofweek,mintemp,maxtemp,rain,label,P,zone_id
0,112,16,2,48.9,64.0,0.0,395,POINT(-73.9841965572521 40.7598175648545),230
1,112,11,2,48.9,64.0,0.0,335,POINT(-73.9815323547465 40.7736331935265),142
2,112,8,2,48.9,64.0,0.0,61,POINT(-73.961763694476 40.8094568126806),166
3,112,14,2,48.9,64.0,0.0,317,POINT(-73.9596350133211 40.7669480510354),141
4,112,6,2,48.9,64.0,0.0,67,POINT(-73.9704426117182 40.749913998467),233


In [0]:
df.drop('P', axis=1, inplace=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 585122 entries, 0 to 585121
Data columns (total 8 columns):
daynumber    585122 non-null int64
hour         585122 non-null int64
dayofweek    585122 non-null object
mintemp      585122 non-null float64
maxtemp      585122 non-null float64
rain         585122 non-null float64
label        585122 non-null int64
zone_id      585122 non-null object
dtypes: float64(3), int64(3), object(2)
memory usage: 35.7+ MB


In [0]:
df["dayofweek"] = df["dayofweek"].astype(int)

In [0]:
df["zone_id"] = df["zone_id"].astype(int)

In [0]:
from pyspark.sql.types import *

mySchema = StructType([ StructField("daynumber", LongType(), True)\
                       ,StructField("hour", IntegerType(), True)\
                       ,StructField("dayofweek", IntegerType(), True)\
                       ,StructField("mintemp", FloatType(), True)\
                       ,StructField("maxtemp", FloatType(), True)\
                       ,StructField("rain", FloatType(), True)\
                       ,StructField("label", IntegerType(), True)\
                       ,StructField("zone_id", IntegerType(), True)
                      ])

In [0]:
py1 = spark.createDataFrame(df,schema=mySchema)

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(spark)

In [29]:
py1.printSchema()

root
 |-- daynumber: long (nullable = true)
 |-- hour: integer (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- mintemp: float (nullable = true)
 |-- maxtemp: float (nullable = true)
 |-- rain: float (nullable = true)
 |-- label: integer (nullable = true)
 |-- zone_id: integer (nullable = true)



In [30]:
py1.show()

+---------+----+---------+-------+-------+----+-----+-------+
|daynumber|hour|dayofweek|mintemp|maxtemp|rain|label|zone_id|
+---------+----+---------+-------+-------+----+-----+-------+
|      112|  16|        2|   48.9|   64.0| 0.0|  395|    230|
|      112|  11|        2|   48.9|   64.0| 0.0|  335|    142|
|      112|   8|        2|   48.9|   64.0| 0.0|   61|    166|
|      112|  14|        2|   48.9|   64.0| 0.0|  317|    141|
|      112|   6|        2|   48.9|   64.0| 0.0|   67|    233|
|      112|  17|        2|   48.9|   64.0| 0.0|  107|     75|
|      112|   0|        2|   48.9|   64.0| 0.0|   46|    239|
|      112|  17|        2|   48.9|   64.0| 0.0|  223|    231|
|      112|  17|        2|   48.9|   64.0| 0.0|  105|    151|
|      112|   9|        2|   48.9|   64.0| 0.0|  169|     68|
|      112|  10|        2|   48.9|   64.0| 0.0|  155|    113|
|      112|  13|        2|   48.9|   64.0| 0.0|  258|    107|
|      112|  15|        2|   48.9|   64.0| 0.0|  172|     79|
|      1

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import RandomForestRegressor

In [0]:
feature=VectorAssembler(inputCols=["daynumber","hour","dayofweek","mintemp","maxtemp","rain","zone_id"],outputCol="independent")

In [0]:
output=feature.transform(py1)

In [36]:
output.show()

+---------+----+---------+-------+-------+----+-----+-------+--------------------+
|daynumber|hour|dayofweek|mintemp|maxtemp|rain|label|zone_id|         independent|
+---------+----+---------+-------+-------+----+-----+-------+--------------------+
|      112|  16|        2|   48.9|   64.0| 0.0|  395|    230|[112.0,16.0,2.0,4...|
|      112|  11|        2|   48.9|   64.0| 0.0|  335|    142|[112.0,11.0,2.0,4...|
|      112|   8|        2|   48.9|   64.0| 0.0|   61|    166|[112.0,8.0,2.0,48...|
|      112|  14|        2|   48.9|   64.0| 0.0|  317|    141|[112.0,14.0,2.0,4...|
|      112|   6|        2|   48.9|   64.0| 0.0|   67|    233|[112.0,6.0,2.0,48...|
|      112|  17|        2|   48.9|   64.0| 0.0|  107|     75|[112.0,17.0,2.0,4...|
|      112|   0|        2|   48.9|   64.0| 0.0|   46|    239|[112.0,0.0,2.0,48...|
|      112|  17|        2|   48.9|   64.0| 0.0|  223|    231|[112.0,17.0,2.0,4...|
|      112|  17|        2|   48.9|   64.0| 0.0|  105|    151|[112.0,17.0,2.0,4...|
|   

In [0]:
final_data=output.select("independent","label")

In [38]:
final_data.show()

+--------------------+-----+
|         independent|label|
+--------------------+-----+
|[112.0,16.0,2.0,4...|  395|
|[112.0,11.0,2.0,4...|  335|
|[112.0,8.0,2.0,48...|   61|
|[112.0,14.0,2.0,4...|  317|
|[112.0,6.0,2.0,48...|   67|
|[112.0,17.0,2.0,4...|  107|
|[112.0,0.0,2.0,48...|   46|
|[112.0,17.0,2.0,4...|  223|
|[112.0,17.0,2.0,4...|  105|
|[112.0,9.0,2.0,48...|  169|
|[112.0,10.0,2.0,4...|  155|
|[112.0,13.0,2.0,4...|  258|
|[112.0,15.0,2.0,4...|  172|
|[112.0,12.0,2.0,4...|  596|
|[112.0,10.0,2.0,4...|  291|
|[112.0,7.0,2.0,48...|  131|
|[112.0,21.0,2.0,4...|  159|
|[112.0,13.0,2.0,4...|  249|
|[112.0,11.0,2.0,4...|  607|
|[112.0,16.0,2.0,4...|  752|
+--------------------+-----+
only showing top 20 rows



In [0]:
train_data,test_data=final_data.randomSplit([0.75,0.25])

In [43]:
type(train_data)

pyspark.sql.dataframe.DataFrame

In [44]:
regressor=LinearRegression(featuresCol="independent",labelCol="label")
regressor.fit(train_data)

LinearRegression_b982fcfa286d

In [48]:
regressor.intercept

AttributeError: ignored

In [0]:
r1=RandomForestRegressor(featuresCol="independent",labelCol="label",numTrees=10, maxDepth=10, seed=42)
model = r1.fit(train_data)

In [70]:
model.featureImportances

SparseVector(7, {0: 0.0048, 1: 0.1923, 2: 0.0254, 3: 0.0028, 4: 0.0031, 5: 0.0018, 6: 0.7698})

In [72]:
type(model)

pyspark.ml.regression.RandomForestRegressionModel

In [71]:
model.transform(test_data).show(10).prediction

+--------------------+-----+------------------+
|         independent|label|        prediction|
+--------------------+-----+------------------+
|[1.0,0.0,3.0,35.0...|    3|19.431874624670495|
|[1.0,0.0,3.0,35.0...|    4|19.431874624670495|
|[1.0,0.0,3.0,35.0...|    1| 17.46552941346361|
|[1.0,0.0,3.0,35.0...|    5|14.278693200504346|
|[1.0,0.0,3.0,35.0...|   83| 66.26396852825638|
|[1.0,0.0,3.0,35.0...|  273| 66.26396852825638|
|[1.0,0.0,3.0,35.0...|    1| 51.95227803952205|
|[1.0,0.0,3.0,35.0...|    8|54.854070032014235|
|[1.0,0.0,3.0,35.0...|  444|54.854070032014235|
|[1.0,0.0,3.0,35.0...|   91|54.854070032014235|
+--------------------+-----+------------------+
only showing top 10 rows



AttributeError: ignored

In [0]:
model.save("/content/drive/My Drive/Data Science/m")

In [75]:
sameModel = RandomForestRegressor.load("/content/drive/My Drive/Data Science/m")

Py4JJavaError: ignored

In [57]:
pred_results=model.evaluate(test_data)

AttributeError: ignored

In [0]:
a=model.transform(test_data)

In [80]:
a.show()

+--------------------+-----+------------------+
|         independent|label|        prediction|
+--------------------+-----+------------------+
|[1.0,0.0,3.0,35.0...|    3|19.431874624670495|
|[1.0,0.0,3.0,35.0...|    4|19.431874624670495|
|[1.0,0.0,3.0,35.0...|    1| 17.46552941346361|
|[1.0,0.0,3.0,35.0...|    5|14.278693200504346|
|[1.0,0.0,3.0,35.0...|   83| 66.26396852825638|
|[1.0,0.0,3.0,35.0...|  273| 66.26396852825638|
|[1.0,0.0,3.0,35.0...|    1| 51.95227803952205|
|[1.0,0.0,3.0,35.0...|    8|54.854070032014235|
|[1.0,0.0,3.0,35.0...|  444|54.854070032014235|
|[1.0,0.0,3.0,35.0...|   91|54.854070032014235|
|[1.0,0.0,3.0,35.0...|    2|54.854070032014235|
|[1.0,0.0,3.0,35.0...|   14|54.854070032014235|
|[1.0,0.0,3.0,35.0...|    2|54.854070032014235|
|[1.0,0.0,3.0,35.0...|    1|54.854070032014235|
|[1.0,0.0,3.0,35.0...|   15|54.854070032014235|
|[1.0,0.0,3.0,35.0...|    3|54.854070032014235|
|[1.0,0.0,3.0,35.0...|   12|54.854070032014235|
|[1.0,0.0,3.0,35.0...|  239|54.854070032